In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime
from lazypredict.Supervised import LazyRegressor

In [2]:
#loading training and testing data
X_train = pd.read_csv('12month_flight_booking.csv')
y_train = pd.read_csv('12months_flight_revenue.csv')
X_test = pd.read_csv('3months_flight_booking_prediction.csv')
y_test = pd.read_csv('3months_flight_revenue_prediction.csv')

In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(195774, 376)
(191712, 376)
(17027, 376)
(16057, 376)


In [4]:
X_train.head()

,Unnamed: 0,Origin,Destination,Cabin,Booking Class,Booking Class Fare USD,Airline,Flight number,Dep_Date,Dep_time,...,Class RDay Res Hold Total RDay: 9,Class RDay Res Hold Total RDay: 8,Class RDay Res Hold Total RDay: 7,Class RDay Res Hold Total RDay: 6,Class RDay Res Hold Total RDay: 5,Class RDay Res Hold Total RDay: 4,Class RDay Res Hold Total RDay: 3,Class RDay Res Hold Total RDay: 2,Class RDay Res Hold Total RDay: 1,Class RDay Res Hold Total RDay: 0
0,0,BBB,DDD,Y,N,200.01,AA,304,2022-04-10,05:55,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10
1,1,BBB,DDD,Y,H,463.65,AA,304,2022-04-10,05:55,...,0.00,0.00,0.00,1.00,6.00,7.00,7.00,7.00,7.00,7
2,2,BBB,DDD,Y,G,149.83,AA,304,2022-04-10,05:55,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1
3,3,BBB,DDD,Y,T,127.29,AA,304,2022-04-10,05:55,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5
4,4,BBB,DDD,Y,S,90.92,AA,304,2022-04-10,05:55,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10


In [5]:
X_train.drop('Unnamed: 0', axis=1, inplace = True)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195774 entries, 0 to 195773
Columns: 375 entries,  Origin to Class RDay Res Hold Total RDay: 0
dtypes: float64(366), int64(2), object(7)
memory usage: 560.1+ MB


In [ ]:
X_train[]